<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dataset_class/fop-core/fop_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 578.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53924 sha256=6d251193f23aa9c8a2b4d3cac43ea4283fe14c53496d65cf67dee77d57dedb5d
  Stored in directory: /root/.cache/pip/wheels/78/ae/2e/5ce789557ca59281a463dabe9d6522e429be240322148e5379
Successfully built docx


In [2]:
!pip install exceptions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement exceptions (from versions: none)
ERROR: No matching distribution found for exceptions


In [13]:
import pandas as pd
import os
import numpy as np
import re
from string import punctuation
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
save_file_csv = 'fop-core2.csv'
file_csv =  'fop-core.csv'
file_dir = '/content/drive/MyDrive/delesmell/dataset_class/fop-core' #Newly stored class address


In [43]:
fileList = []
def file_name(file_dir):
    for root, dirs, files in os.walk(file_dir):
#       print(root) #current directory path
#       print(files) #All non-directory subfiles in the current path
         for file in files:
            if file[-4:] == '.txt':
                fileList.append(os.path.join(root, file))

In [44]:
file_name(file_dir)

In [45]:
fileList

['/content/drive/MyDrive/delesmell/dataset_class/fop-core/PDFResourcesTestCase.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/ListProperty.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/OCAColor.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/PostscriptParser.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/StreamedResourceGroup.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/Java2DPainter.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/SimplePageMaster.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/Type1FontLoader.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/PDFAttachmentTestCase.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/PDFPages.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/PCLTTFFontReader.txt',
 '/content/drive/MyDrive/delesmell/dataset_class/fop-core/MultiSwitchLayoutManager.txt',
 '/content/drive/MyDrive/deles

In [46]:
# stopwords
stopwords = ['abstract', 'java','assert', 'boolean', 'break', 'byte', 'case', 'catch', 'class','char', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extends', 'final', 'finally', 'float', 'for', 'if', 'implements', 'import', 'int', 'interface', 'long', 'native', 'new', 'package','private','public', 'protected',  'return', 'short', 'static', 'strictfp', 'super', 'switch', 'synchronized', 'this', 'throw', 'transient', 'try', 'void', 'volatile', 'while']
dicts = {'\n': '','!': '', '"': '','：':'', '#': '', '$': '', '%': '', '&': '', "'": '', '(': '', ')': '', '*': '', '+': '', ',': '', '-': '', '.': '', '/': '', ':': '', ';': '', '<': '', '=': '', '>': '', '?': '', '@': '', '[': '', '\\': '', ']': '', '^': '', '_': '', '`': '', '{': '', '|': '', '}': '', '~': ''}
# stopwords = {'abstract':'', 'java':'','assert':'', 'boolean':'', 'break':'', 'byte':'', 'case':'', 'catch':'', 'class':'','char':'', 'continue':'', 'default':'', 'do':'', 'double':'', 'else':'', 'enum':'', 'extends':'', 'final':'', 'finally':'', 'float':'', 'for':'', 'if':'', 'implements':'', 'import':'', 'int':'', 'interface':'', 'long':'', 'native':'', 'new':'', 'package':'','private':'','public':'', 'protected':'',  'return':'', 'short':'', 'static':'', 'strictfp':'', 'super':'', 'switch':'', 'synchronized':'', 'this':'', 'throw':'', 'transient':'', 'try':'','void':'', 'volatile':'', 'while':''}

In [47]:
def lm_find_unchinese(line):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    unchinese = re.sub(pattern,"",line) #排除汉字
    unchinese = re.sub('[{}]'.format(punctuation),"",unchinese) #排除中文符号
    #print("unchinese:",unchinese)
    return unchinese

In [48]:
def del_stopwords(line):
#     for line in file.readlines():
#     dicts = {i: '' for i in punctuation}
    punc_table = str.maketrans(dicts)
    new_line = line.translate(punc_table)

#     for word in stopwords:
#           new_line = new_line.replace(word, '')

    return new_line


In [49]:
index = []
index_all = []
class_name = []
Brain_Class = []
for i, f in enumerate(fileList):
    with open(f,encoding='gb18030',errors='ignore') as file:
        list = []
        list2 = []
        is_brain = 0
        filename = os.path.basename(file.name)
        name = filename[:-4]
        class_name.append(name)
        list.append(name)
        list2.append(name)
        
        for line in file.readlines():
            line = lm_find_unchinese(line)
            line = del_stopwords(line)
            values = line.split(' ')
            coefs = values[1:-1]
            coefs = [i for i in coefs if i != '']
            keyword_brain = 'hongshuai'            

            if  keyword_brain in line:
                is_brain = 1
#             if len(coefs) != 0:
#                 if coefs != ' ':
#                     if coefs != '\n':
            for i in coefs:
                list.append(i)

        index.append(list)
        
        if is_brain == 0:
            list2.append(False)
        else :
            list2.append(True)
            print('T')
        
#         print (list2)
        
        Brain_Class.append(list2)
            


T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T


In [50]:
for k in range(0,len(Brain_Class)):
    a = Brain_Class[k][0]
    if '_' in a:
        n = a.find('_')
        new_str = ''
        for i in range(0, n):
            new_str = new_str + a[i]
        Brain_Class[k][0] = new_str
    print(Brain_Class[k][0])

PDFResourcesTestCase
ListProperty
OCAColor
PostscriptParser
StreamedResourceGroup
Java2DPainter
SimplePageMaster
Type1FontLoader
PDFAttachmentTestCase
PDFPages
PCLTTFFontReader
MultiSwitchLayoutManager
CharactersetEncoderTestCase
PropertyMaker
URIResolverWrapperTestCase
LazyFont
KnuthAlgorithmTestCase
AFPPainterTestCase
LengthRangeProperty
Java2DRenderingSettings
TableCellLayoutManager
PCLTTFTableFactory
RegionEnd
CharUtilitiesTestCase
PaddingElement
PDFVTTestCase
RepeatablePageMasterReference
Java2DRenderer
BidiResolver
PDFDeviceColorSpace
PDFFlowExtTextPainter
Glyphs
UnicodeClasses
PDFGoTo
ImageEncoderCCITTFax
BitmapImage
TableRow
ImageProxyPanel
SimpleSVGUserAgent
MinOptMaxTestCase
RtfTable
PCLRendererContext
AbstractBitmapDocumentHandler
PSPageTrailerCodeBefore
RasterFont
ImageAdapter
TableCaption
SingleByteFont
SpaceVal
PDFStreamTestCase
GlyphSubstitution
Wrapper
Processor
MissingLanguageWarningTestCase
BitmapImageUtilTestCase
GlyphSubstitutionSubtable
PCLGenerator
RtfFootnote
PCL

In [51]:
Brain_Class = np.array(Brain_Class)
Brain_Class

array([['PDFResourcesTestCase', 'False'],
       ['ListProperty', 'False'],
       ['OCAColor', 'False'],
       ...,
       ['PDFPainterTestCase', 'False'],
       ['RtfStructureException', 'False'],
       ['PDFParentTree', 'False']], dtype='<U41')

In [52]:
len(index)

1650

In [53]:
def additem(list):
    if len(list)<3:
        q = len(list)-1
        str = '_1'
        str2 = '_2'
        item = ''.join([list[q], str])
        item_2 = ''.join([list[q], str2])
        # item = list[q]
        list.append(item)
        list.append(item_2)
        # print(list)
    return list

for item in index_all:
    # print(item)
    item = additem(item)
    # print(item)

In [54]:
news_df = np.array(index,dtype=object)
print(news_df)

[list(['PDFResourcesTestCase', 'PDFColorSpace', 'PDFFunction', 'return', 'new', 'PDFFunction', 'final', 'Double', 'final', 'Double', 'ListDouble', 'domainArraysasListnew', 'ListDouble', 'rangeArraysasListnew', 'float', 'cZeronew', 'float', 'PDFFunction', 'tintFunctionnew', 'return', 'PDFPattern', 'createPDFPatternPDFResources', 'resPDFDocument', 'ListDouble', 'bboxnew', 'ListDouble', 'theMatrixnew', 'for', 'int', 'i0', 'i', '6', 'i', 'PDFPattern', 'patternnew', 'PDFPatternres111bbox11theMatrixnullnew', 'patternsetNamePa', 'return', 'PDFShading', 'createPDFShadingPDFResources', 'resPDFDocument', 'ListDouble', 'coordsnew', 'PDFFunction', 'PDFDeviceColorSpace', 'deviceColorspacenew', 'PDFShading', 'shadingnew', 'shadingsetNameSh', 'return', 'Test', 'PDF', 'resources', 'output', 'with', 'color', 'space', 'pattern', 'and', 'shading', 'if', 'the', 'PDF', 'resource', 'object', 'has', 'a', 'parent', 'resource', 'throws', 'public', 'void', 'testOutputWithParent', 'throws', 'IOException', 'PDFDo

In [55]:
# # 训练阶段
def f_lsa(news_df, n):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(news_df)
    # print(vectorizer.get_feature_names())# 获得模型直接分析数据找到的词汇量（上面单词的集合）
    # print(X.toarray())# 直接打印X输出的是每个词的位置
    #     print(X)

    n_cap = 1
    svd_model = TruncatedSVD(n_components=n_cap)  # 潜在语义分析，设置4个话题
    X2 = svd_model.fit_transform(X)  # 训练并进行转化

    # print("--------lsa奇异值---------")
    # print(svd_model.singular_values_)
    # print("--------文本在话题向量空间下的表示 左奇异矩阵---------")
    # print(X2)

    terms = vectorizer.get_feature_names()
    # print("--------话题及关键词---------")
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:7]
        # print("Topic " + str(i) + ": ")
        for t in sorted_terms:
            print(t[0])
    print(" ")

    D = svd_model.components_
    T = X2
    S = np.diagflat(svd_model.singular_values_)

    X = np.dot(T, S)
    X = np.dot(X, D)
    c = np.dot(X, X.T)

    acsm = None
    sum = 0
    for i in c:
        for j in i:
            if j < 0:
                j = 0
            sum += j
        acsm = sum / n / n
    # print("-------------acsm----------------------")
    # print(acsm)

    return acsm

In [56]:
q = 0
acsm= []
for i in index:
    lists = []
    print("*************",i,"****************")
    # print("------------------------------------------------------------------")
    for j in i:
        lists.append(j)
    # lists = deleteDuplicatedElementFromList(lists)
    if len(lists) > 1:
        acsm.append(f_lsa(lists,len(i)))
    else :
        acsm.append(1)
#

# print(type(class_name))
# print(type(acsm))

************* ['PDFResourcesTestCase', 'PDFColorSpace', 'PDFFunction', 'return', 'new', 'PDFFunction', 'final', 'Double', 'final', 'Double', 'ListDouble', 'domainArraysasListnew', 'ListDouble', 'rangeArraysasListnew', 'float', 'cZeronew', 'float', 'PDFFunction', 'tintFunctionnew', 'return', 'PDFPattern', 'createPDFPatternPDFResources', 'resPDFDocument', 'ListDouble', 'bboxnew', 'ListDouble', 'theMatrixnew', 'for', 'int', 'i0', 'i', '6', 'i', 'PDFPattern', 'patternnew', 'PDFPatternres111bbox11theMatrixnullnew', 'patternsetNamePa', 'return', 'PDFShading', 'createPDFShadingPDFResources', 'resPDFDocument', 'ListDouble', 'coordsnew', 'PDFFunction', 'PDFDeviceColorSpace', 'deviceColorspacenew', 'PDFShading', 'shadingnew', 'shadingsetNameSh', 'return', 'Test', 'PDF', 'resources', 'output', 'with', 'color', 'space', 'pattern', 'and', 'shading', 'if', 'the', 'PDF', 'resource', 'object', 'has', 'a', 'parent', 'resource', 'throws', 'public', 'void', 'testOutputWithParent', 'throws', 'IOException'

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Streaming output truncated to the last 5000 lines.
 
************* ['EnvironmentalProfileFactory', 'Creates', 'the', 'default', 'environment', 'that', 'FOP', 'is', 'invoked', 'in', 'This', 'default', 'profile', 'has', 'no', 'operational', 'restrictions', 'for', 'param', 'defaultBaseUri', 'the', 'default', 'base', 'URI', 'for', 'resolving', 'resource', 'param', 'resourceResolver', 'the', 'resource', 'return', 'the', 'environment', 'static', 'EnvironmentProfile', 'createDefaultURI', 'defaultBaseUriResourceResolver', 'return', 'new', 'ProfiledefaultBaseUriresourceResolvercreateFontManagerdefaultBaseUriresourceResolverFontDetectorFactorycreateDefaultFontCacheManagerFactorycreateDefaultnew', 'static', 'FontManager', 'createFontManagerURI', 'defaultBaseUriResourceResolver', 'resourceResolverFontDetector', 'fontDetectorFontCacheManager', 'InternalResourceResolver', 'return', 'new', 'Creates', 'an', 'IOrestricted', 'environment', 'for', 'FOP', 'by', 'disabling', 'some', 'of', 'the', 'environme

In [57]:
c = []
for i in range(len(class_name)):
      for j in range(len(acsm)):
        if i==j:
            t=[class_name[i],acsm[j]]
            c.append(t)

# c = list(zip(class_name,acsm))
# print(type(c))
c = np.array(c)
# print(c)
# print(c[0][0])

In [65]:
df =pd.read_csv("/content/drive/MyDrive/delesmell/dataset_class/fop-core/fop-core.csv") 

In [66]:
x= df.iloc[:, 0]

In [67]:
c

array([['PDFResourcesTestCase', '0.013032090985891429'],
       ['ListProperty', '0.10204568591469719'],
       ['OCAColor', '0.1975672291165484'],
       ...,
       ['PDFPainterTestCase', '0.02764135346814655'],
       ['RtfStructureException', '0.12500000000000025'],
       ['PDFParentTree', '0.015344493659117367']], dtype='<U41')

In [68]:
Brain_Class

array([['PDFResourcesTestCase', 'False'],
       ['ListProperty', 'False'],
       ['OCAColor', 'False'],
       ...,
       ['PDFPainterTestCase', 'False'],
       ['RtfStructureException', 'False'],
       ['PDFParentTree', 'False']], dtype='<U41')

In [69]:
for i in range(0,len(Brain_Class)):
    if 'False' == Brain_Class[i][1]:
        q = 1
    else :
        print('*')

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


In [71]:

ngList=[]

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(c[i][0])
        st = st[1:]
#         print(st)
        if index == st:
            ngList.append(c[i][1])
            q = 0
            break
#             print('*')
    if q==1 :
        ngList.append(1)
        

In [72]:
brainList=[] 

for index in x:
#     print(index)
    q = 1
    for i in range(0,len(c)):
        st = str(Brain_Class[i][0])
        st = st[1:]
        if index == st:
            brainList.append(Brain_Class[i][1])
            q = 0
            break
    if q==1 :
        brainList.append('0')
        
        
print(len(brainList))
print(len(x))

2847
2847


In [73]:
column=df.columns.tolist()

In [74]:
df['acsm'] = ngList 
df['is_brain'] = brainList 
df.to_csv(save_file_csv,mode = 'a',index =False)

In [75]:
df.head()

,NAME,AMW,ATDF,BOVM,BUR,CBO,CC,CM,CRIX,DAC,...,NOA,NOAM,NOD,NOM,NOPA,TCC,WMC,WOC,acsm,is_brain
0,GenerateBidiClass,7.91,3,0,1.0,3,0,0,2.5,2,...,6,0,0,11,0,0.09,87,1.0,1,0
1,GenerateBidiTestData,5.09,0,0,1.0,2,0,0,2.5,0,...,13,0,0,56,0,0.05,285,1.0,1,0
2,BidiClass,3.00,0,0,1.0,0,0,0,0.0,2,...,5,0,0,3,0,0.00,9,1.0,1,0
3,BidiResolver,2.50,3,0,1.0,6,0,0,0.0,0,...,1,0,0,16,0,0.10,40,1.0,1,0
4,TextInterval,2.36,1,0,1.0,3,1,4,0.0,1,...,5,6,0,14,0,0.02,33,1.0,1,0
